# example of using ImageDataGenerator

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### load dataset

In [2]:
(trainX, trainY), (testX, testY) = mnist.load_data()

#### reshape dataset to have a single channel

In [3]:
width, height, channels = trainX.shape[1], trainX.shape[2], 1
trainX = trainX.reshape((trainX.shape[0], width, height, channels))
testX = testX.reshape((testX.shape[0], width, height, channels))

#### one hot encode target values

In [4]:
trainY = to_categorical(trainY)
testY = to_categorical(testY)

#### confirm scale of pixels

In [5]:
print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))

Train min=0.000, max=255.000
Test min=0.000, max=255.000


#### create generator

In [40]:
# https://keras.io/preprocessing/image/#imagedatagenerator-class
#datagen = ImageDataGenerator(rescale=1.0/255.0)
#datagen = ImageDataGenerator(featurewise_center=True)
datagen = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True,
                             zoom_range=0.1)

#### fit the generator

In [43]:
datagen.fit(trainX)

#### prepare an iterators to preprocess images

In [44]:
train_iterator = datagen.flow(trainX, trainY, batch_size=64)
test_iterator = datagen.flow(testX, testY, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

Batches train=938, test=157


#### confirm the scaling works

In [45]:
batchX, batchy = train_iterator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(64, 28, 28, 1), min=-0.424, max=2.821


#### define model

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

#### compile model

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### fit model with generator

In [11]:
model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)

Epoch 1/5
938/938 [==============================] - 45s 48ms/step - loss: 0.1793 - accuracy: 0.9462
Epoch 2/5
938/938 [==============================] - 46s 49ms/step - loss: 0.0518 - accuracy: 0.9846
Epoch 3/5
938/938 [==============================] - 47s 50ms/step - loss: 0.0370 - accuracy: 0.9883
Epoch 4/5
938/938 [==============================] - 44s 47ms/step - loss: 0.0279 - accuracy: 0.9912
Epoch 5/5
938/938 [==============================] - 44s 47ms/step - loss: 0.0219 - accuracy: 0.9929


#### evaluate model

In [12]:
_, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

Test Accuracy: 98.960
